# BaseLine

- 참가자 분들의 이해를 돕기위한 baseline 입니다.
- 해당 코드는 이해를 위한 참고용입니다.
- 반드시 결과는 해당코드 아래의 **제출 Predict** 부분을 확인하시고 해당 방법처럼 제출해 주시면 감사하겠습니다.

&nbsp;

In [8]:
import os
import warnings
warnings.filterwarnings("ignore")
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras import backend as K
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
import threading
import random
import rasterio
import os
import numpy as np
import sys
from sklearn.utils import shuffle as shuffle_lists
from keras.models import *
from keras.layers import *
import numpy as np
from keras import backend as K
from sklearn.model_selection import train_test_split
import joblib

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (C:\Users\kkong\anaconda3\Lib\site-packages\keras\preprocessing\image\__init__.py)

&nbsp;

## 사용할 함수 정의

In [ ]:
MAX_PIXEL_VALUE = 65535 # 이미지 정규화를 위한 픽셀 최대값

class threadsafe_iter:
    """
    데이터 불러올떼, 호출 직렬화
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()


def threadsafe_generator(f):
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))

    return g

def get_img_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    img = np.float32(img)/MAX_PIXEL_VALUE

    return img

def get_img_762bands(path):
    img = rasterio.open(path).read((7,6,2)).transpose((1, 2, 0))
    img = np.float32(img)/MAX_PIXEL_VALUE

    return img

def get_mask_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    seg = np.float32(img)
    return seg



@threadsafe_generator
def generator_from_lists(images_path, masks_path, batch_size=32, shuffle = True, random_state=None, image_mode='10bands'):

    images = []
    masks = []

    fopen_image = get_img_arr
    fopen_mask = get_mask_arr

    if image_mode == '762':
        fopen_image = get_img_762bands

    i = 0
    # 데이터 shuffle
    while True:

        if shuffle:
            if random_state is None:
                images_path, masks_path = shuffle_lists(images_path, masks_path)
            else:
                images_path, masks_path = shuffle_lists(images_path, masks_path, random_state= random_state + i)
                i += 1


        for img_path, mask_path in zip(images_path, masks_path):

            img = fopen_image(img_path)
            mask = fopen_mask(mask_path)
            images.append(img)
            masks.append(mask)

            if len(images) >= batch_size:
                yield (np.array(images), np.array(masks))
                images = []
                masks = []

In [1]:
def FCN(nClasses, input_height=128, input_width=128, n_filters=16, dropout=0.1, batchnorm=True):

    img_input = Input(shape=(input_height, input_width, 3))

    x = Conv2D(n_filters, (3, 3), activation='relu', padding='same', name='conv1')(img_input)
    x = Conv2D(n_filters, (3, 3), activation='relu', padding='same', name='conv2')(x)
    f1 = x

    x = Conv2D(n_filters, (3, 3), activation='relu', padding='same', name='conv3')(x)
    x = Conv2D(n_filters, (3, 3), activation='relu', padding='same', name='conv4')(x)
    f2 = x

    o = Conv2D(nClasses, (3, 3), activation='relu', padding='same', name="out")(x)

    model = Model(img_input, o)

    return model

In [ ]:
# 두 샘플 간의 유사성 metric
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

# 픽셀 정확도를 계산 metric
def pixel_accuracy (y_true, y_pred):
    sum_n = np.sum(np.logical_and(y_pred, y_true))
    sum_t = np.sum(y_true)

    if (sum_t == 0):
        pixel_accuracy = 0
    else:
        pixel_accuracy = sum_n / sum_t
    return pixel_accuracy

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, MaxPooling2D, Concatenate

def conv_block(input_tensor, filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding)(input_tensor)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    return x

def yolo_model(input_shape, n_classes):
    input_layer = Input(shape=input_shape)

    # Down-sampling blocks
    x = conv_block(input_layer, 16)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = conv_block(x, 32)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = conv_block(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = conv_block(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Bottleneck block
    x = conv_block(x, 256)
    
    # Up-sampling blocks
    x = Conv2D(128, (1, 1), padding='same')(x)
    x = Conv2D(64, (1, 1), padding='same')(x)
    x = Conv2D(32, (1, 1), padding='same')(x)
    x = Conv2D(16, (1, 1), padding='same')(x)

    # Output layer
    output_layer = Conv2D(n_classes, (1, 1), activation='softmax', padding='same')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

def get_yolo_model(input_shape=(256, 256, 3), n_classes=2):
    return yolo_model(input_shape, n_classes)

&nbsp;

## parameter 설정

In [ ]:
# 사용할 데이터의 meta정보 가져오기

train_meta = pd.read_csv('/data2/local_datasets/fire/train_meta.csv')
test_meta = pd.read_csv('/data2/local_datasets/fire/test_meta.csv')


# 저장 이름
save_name = 'fcn_baseline'

N_FILTERS = 16 # 필터수 지정
N_CHANNELS = 3 # channel 지정
EPOCHS = 50 # 훈련 epoch 지정
BATCH_SIZE = 64 # batch size 지정
IMAGE_SIZE = (256, 256) # 이미지 크기 지정
MODEL_NAME = 'FCN' # 모델 이름
RANDOM_STATE = 42 # seed 고정
INITIAL_EPOCH = 0 # 초기 epoch

# 데이터 위치
IMAGES_PATH = '/data2/local_datasets/fire/train_img/'
MASKS_PATH = '/data2/local_datasets/fire/train_mask/'

# 가중치 저장 위치
OUTPUT_DIR = '/data2/local_datasets/fire/train_output/'
WORKERS = 4

# 조기종료
EARLY_STOP_PATIENCE = 5

# 중간 가중치 저장 이름
CHECKPOINT_PERIOD = 5
CHECKPOINT_MODEL_NAME = 'checkpoint-{}-{}-epoch_{{epoch:02d}}.hdf5'.format(MODEL_NAME, save_name)

# 최종 가중치 저장 이름
FINAL_WEIGHTS_OUTPUT = 'model_{}_{}_final_weights.h5'.format(MODEL_NAME, save_name)

# 사용할 GPU 이름
CUDA_DEVICE = 3


# 저장 폴더 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# GPU 설정
os.environ["CUDA_VISIBLE_DEVICES"] = str(CUDA_DEVICE)
try:
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    K.set_session(sess)
except:
    pass

try:
    np.random.bit_generator = np.random._bit_generator
except:
    pass


# train : val = 8 : 2 나누기
x_tr, x_val = train_test_split(train_meta, test_size=0.2, random_state=RANDOM_STATE)
print(len(x_tr), len(x_val))

26860 6715


In [ ]:
# train : val 지정 및 generator
images_train = [os.path.join(IMAGES_PATH, image) for image in x_tr['train_img'] ]
masks_train = [os.path.join(MASKS_PATH, mask) for mask in x_tr['train_mask'] ]

images_validation = [os.path.join(IMAGES_PATH, image) for image in x_val['train_img'] ]
masks_validation = [os.path.join(MASKS_PATH, mask) for mask in x_val['train_mask'] ]

train_generator = generator_from_lists(images_train, masks_train, batch_size=BATCH_SIZE, random_state=RANDOM_STATE, image_mode="762")
validation_generator = generator_from_lists(images_validation, masks_validation, batch_size=BATCH_SIZE, random_state=RANDOM_STATE, image_mode="762")


# model 불러오기
model = FCN(nClasses=1, input_height=IMAGE_SIZE[0], input_width=IMAGE_SIZE[1], n_filters=N_FILTERS, n_channels=N_CHANNELS)
model.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()


# checkpoint 및 조기종료 설정
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=EARLY_STOP_PATIENCE)
checkpoint = ModelCheckpoint(os.path.join(OUTPUT_DIR, CHECKPOINT_MODEL_NAME), monitor='loss', verbose=1,
save_best_only=True, mode='auto', period=CHECKPOINT_PERIOD)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

&nbsp;

## model 훈련

In [ ]:
print('---model 훈련 시작---')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(images_train) // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=len(images_validation) // BATCH_SIZE,
    callbacks=[checkpoint, es],
    epochs=EPOCHS,
    workers=WORKERS,
    initial_epoch=INITIAL_EPOCH
)
print('---model 훈련 종료---')

---model 훈련 시작---
Epoch 1/50
419/419 [==============================] - 541s 1s/step - loss: 0.1251 - accuracy: 0.9952 - val_loss: 0.0244 - val_accuracy: 0.9997
Epoch 2/50
419/419 [==============================] - 343s 819ms/step - loss: 0.0160 - accuracy: 0.9997 - val_loss: 0.0082 - val_accuracy: 0.9997
Epoch 3/50
419/419 [==============================] - 342s 819ms/step - loss: 0.0054 - accuracy: 0.9998 - val_loss: 0.0035 - val_accuracy: 0.9999
Epoch 4/50
419/419 [==============================] - 343s 819ms/step - loss: 0.0027 - accuracy: 0.9999 - val_loss: 0.0020 - val_accuracy: 0.9999
Epoch 5/50
419/419 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 0.9999
Epoch 5: loss improved from inf to 0.00141, saving model to ./train_output\checkpoint-unet-base_line-epoch_05.hdf5
419/419 [==============================] - 345s 825ms/step - loss: 0.0014 - accuracy: 0.9999 - val_loss: 0.0010 - val_accuracy: 0.9999
Epoch 6/50
419/419 [==============================] - 3

&nbsp;

## model save

In [ ]:
print('가중치 저장')
model_weights_output = os.path.join(OUTPUT_DIR, FINAL_WEIGHTS_OUTPUT)
model.save_weights(model_weights_output)
print("저장된 가중치 명: {}".format(model_weights_output))

Saving weights
Weights Saved: ./train_output/model_unet_base_line_final_weights.h5


## inference

- 학습한 모델 불러오기

In [ ]:
model = FCN(nClasses=1, input_height=IMAGE_SIZE[0], input_width=IMAGE_SIZE[1], n_filters=N_FILTERS, n_channels=N_CHANNELS)
model.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

model.load_weights('./train_output/model_unet_base_line_final_weights.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 256, 256, 16  64         ['conv2d_19[0][0]']              
 ormalization)                  )                                                           

## 제출 Predict
- numpy astype uint8로 지정
- 반드시 pkl로 저장


In [ ]:
y_pred_dict = {}

for i in test_meta['test_img']:
    img = get_img_762bands(f'./test_img/{i}')
    y_pred = model.predict(np.array([img]), batch_size=1)

    y_pred = np.where(y_pred[0, :, :, 0] > 0.25, 1, 0) # 임계값 처리
    y_pred = y_pred.astype(np.uint8)
    y_pred_dict[i] = y_pred

joblib.dump(y_pred_dict, './y_pred.pkl')

1/1 [==============================] - 0s 15ms/step
